## Load Packages

In [1]:
        
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# Predictions from an AR(1)

Consider an AR(1) with mean $\mu$

$r_{t+1}-\mu=\rho\left(  r_{t}-\mu\right)  +\varepsilon_{t+1}$

The forecast for $t+s$ is

$\text{E}_{t}r_{t+s}=\left(  1-\rho^{s}\right)  \mu+\rho ^{s}r_{t}$


In [3]:
function AR1Prediction(x0,ρ,μ,s)                       #function for calculating forecast from AR(1)
    E0xs = (1-ρ^s)*μ + ρ^s*x0
    return E0xs
end

AR1Prediction (generic function with 1 method)

In [4]:
sMax = 120
sM = 1:sMax

ρ = 0.975
μ = 0.05
xPred = fill(NaN,(sMax,3))                              #predictions[horizon,today's value]
for s in sM
    xPred[s,:] = AR1Prediction([0.03 0.05 0.07],ρ,μ,s)
end    

plot(sM,xPred[:,1],color=:black,linewidth=2,label="3%")
plot!(sM,xPred[:,2],color=:red,linewidth=2,label="5%")
plot!(sM,xPred[:,3],color=:blue,linewidth=2,label="7%")
title!("Forecast of future short interest rates (based on different starting values), (mu,rho) = ($μ,$ρ)"),
xlabel!("forecast horizons (months)")
ylabel!("short interest rate")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 Forecast of future short interest rates (based on different starting values), (mu,rho) = (0.05,0.975) 
 
 
 forecast horizons (months) 
 
 
 short interest rate 
 
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,368.504 44.0151,363.996 48.6601,359.601 53.3051,355.316 57.9501,351.137 62.5951,347.064 67.2401,343.092 71.8851,339.219 76.5301,335.444 81.1751,331.762 
 85.8202,328.173 90.4652,324.673 95.1102,321.261 99.7552,317.934 104.4,314.691 109.045,311.528 113.69,308.445 118.335,305.438 122.98,302.507 127.625,299.649 
 132.27,296.863 136.915,294.146 141.56,291.497 146.205,288.914 150.85,286.396 155.495,283.941 160.14,281.547 164.785,279.213 169.43,276.937 174.075,274.719 
 178.72,272.556 183.365,270.446 188.01,268.39 192.655,266.385 197.3,264.43 201.945,262.524 206.59,260.666 211.235,258.854 215.88,257.087 220.525,255.365 
 225.17,253.685 229.815,252.048 234.46,250.451 239.105,248.895 243.75,247.377 248.395,245.897 253.04,244.455 257.685,243.048 262.33,241.677 266.975,240.339 
 271.62,239.036 276.265,237.764 280.91,236.525 285.555,235.317 290.2,234.138 294.845,232.99 299.491,231.87 304.136,230.778 308.781,229.713 313.426,228.675 
 318.071,227.663 322.716,226.676 327.361,225.714 332.006,224.776 336.651,223.861 341.296,222.969 345.941,222.1 350.586,221.252 355.231,220.425 359.876,219.619 
 364.521,218.834 369.166,218.068 373.811,217.321 378.456,216.592 383.101,215.882 387.746,215.19 392.391,214.515 397.036,213.857 401.681,213.215 406.326,212.589 
 410.971,211.979 415.616,211.385 420.261,210.805 424.906,210.239 429.551,209.688 434.196,209.151 438.841,208.627 443.486,208.116 448.131,207.617 452.776,207.132 
 457.421,206.658 462.066,206.196 466.711,205.746 471.356,205.307 476.001,204.879 480.646,204.462 485.291,204.055 489.936,203.659 494.581,203.272 499.226,202.895 
 503.871,202.527 508.516,202.169 513.161,201.819 517.806,201.478 522.451,201.146 527.096,200.822 531.741,200.506 536.386,200.198 541.031,199.898 545.676,199.606 
 550.321,199.32 554.966,199.042 559.611,198.771 564.256,198.506 568.901,198.248 573.546,197.997 578.191,197.751 582.836,197.512 587.481,197.279 592.126,197.052 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,188.189 44.0151,188.189 48.6601,188.189 53.3051,188.189 57.9501,188.189 62.5951,188.189 67.2401,188.189 71.8851,188.189 76.5301,188.189 81.1751,188.189 
 85.8202,188.189 90.4652,188.189 95.1102,188.189 99.7552,188.189 104.4,188.189 109.045,188.189 113.69,188.189 118.335,188.189 122.98,188.189 127.625,188.189 
 132.27,188.189 136.915,188.189 141.56,188.189 146.205,188.189 150.85,188.189 155.495,188.189 160.14,188.189 164.785,188.189 169.43,188.189 174.075,188.189 
 178.72,188.189 183.365,188.189 188.01,188.189 192.655,188.189 197.3,188.189 201.945,188.189 206.59,188.189 211.235,188.189 215.88,188.189 220.525,188.189 
 225.17,188.189 229.815,188.189 234.46,188.189 239.105,188.189 243.75,188.189 248.395,188.189 253.04,188.189 257.685,188.189 262.33,188.189 266.975,188.189 
 271.62,188.189 276.265,188.189 280.91,188.189 285.555,188.189 290.2,188.189 294.845,188.189 299.491,188.189 304.136,188.189 308.781,188.189 313.426,188.189 
 318.071,188.189 322.716,188.189 327.361,188.189 332.006,188.189 336.651,188.189 341.296,188.189 345.941,188.189 350.586,188.189 355.231,188.189 359.876,188.189 
 364.521,188.189 369.166,188.189 373.811,188.189 378.456,188.189 383.101,188.189 387.746,188.189 392.391,188.189 397.036,188.189 401.681,188.189 406.326,188.189 
 410.971,188.189 415.616,188.189 420.261,188.189 424.906,188.189 429.551,188.189 434.196,188.189 438.841,188.189 443.486,188.189 448.131,188.189 452.776,188.189 
 457.421,188.189 462.066,188.189 466.711,188.189 471

# The Vasicek Model

The Vasicek model (without risk premia) implies that 

$\begin{align}
y_{t}(m)  &  =a(m)+b(m)r_{t} \ \text{, where } \\
a(m)  &  = \mu\left[  1-b(m)\right] \ \text{ and}\\
b(m)  &  =(1-\rho^{m})/[(1-\rho)m].
\end{align}$

In [5]:
function Vasicek(r,ρ,μ,m)                   #Vasicek model -> interest rate and (a,b) coeffs
    if ρ == 1.0
        b = 1.0
    else    
        b = (1-ρ^m)/((1-ρ)*m)
    end    
    a = μ*(1-b)
    y = a + b*r
    return y,a,b
end

Vasicek (generic function with 1 method)

In [6]:
yM = fill(NaN,(sMax,3))                             #interest rate[maturity,today's short rate]
ab = fill(NaN,(sMax,2))
for m in sM
    (y,a,b) = Vasicek([0.03 0.05 0.07],ρ,μ,m)
    yM[m,:] = y
    ab[m,:] = [a b]
end    

In [8]:
plot(sM,yM[:,1],color=:black,linewidth=2,label="3%")
plot!(sM,yM[:,2],color=:red,linewidth=2,label="5%")
plot!(sM,yM[:,3],color=:blue,linewidth=2,label="7%")
title!("Vasicek model: interest rates (for different current short rates), (mu,rho) = ($μ,$ρ)"),
xlabel!("maturity (months)")
ylabel!("interest rate")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 Vasicek model: interest rates (for different current short rates), (mu,rho) = (0.05,0.975) 
 
 
 maturity (months) 
 
 
 interest rate 
 
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,368.504 44.0151,366.25 48.6601,364.034 53.3051,361.854 57.9501,359.711 62.5951,357.603 67.2401,355.53 71.8851,353.491 76.5301,351.486 81.1751,349.513 
 85.8202,347.573 90.4652,345.665 95.1102,343.788 99.7552,341.941 104.4,340.124 109.045,338.337 113.69,336.579 118.335,334.849 122.98,333.147 127.625,331.472 
 132.27,329.824 136.915,328.202 141.56,326.606 146.205,325.036 150.85,323.49 155.495,321.969 160.14,320.472 164.785,318.998 169.43,317.548 174.075,316.12 
 178.72,314.715 183.365,313.332 188.01,311.97 192.655,310.629 197.3,309.309 201.945,308.009 206.59,306.73 211.235,305.47 215.88,304.229 220.525,303.008 
 225.17,301.805 229.815,300.62 234.46,299.453 239.105,298.304 243.75,297.173 248.395,296.058 253.04,294.96 257.685,293.878 262.33,292.813 266.975,291.764 
 271.62,290.73 276.265,289.711 280.91,288.708 285.555,287.719 290.2,286.745 294.845,285.785 299.491,284.839 304.136,283.907 308.781,282.988 313.426,282.083 
 318.071,281.191 322.716,280.312 327.361,279.445 332.006,278.591 336.651,277.749 341.296,276.919 345.941,276.101 350.586,275.294 355.231,274.499 359.876,273.715 
 364.521,272.942 369.166,272.18 373.811,271.428 378.456,270.687 383.101,269.956 387.746,269.236 392.391,268.525 397.036,267.824 401.681,267.133 406.326,266.451 
 410.971,265.779 415.616,265.115 420.261,264.461 424.906,263.816 429.551,263.179 434.196,262.551 438.841,261.931 443.486,261.319 448.131,260.716 452.776,260.12 
 457.421,259.533 462.066,258.953 466.711,258.381 471.356,257.816 476.001,257.259 480.646,256.709 485.291,256.166 489.936,255.631 494.581,255.102 499.226,254.58 
 503.871,254.064 508.516,253.556 513.161,253.053 517.806,252.557 522.451,252.068 527.096,251.584 531.741,251.107 536.386,250.636 541.031,250.17 545.676,249.71 
 550.321,249.256 554.966,248.808 559.611,248.365 564.256,247.928 568.901,247.496 573.546,247.069 578.191,246.648 582.836,246.231 587.481,245.82 592.126,245.413 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,188.189 44.0151,188.189 48.6601,188.189 53.3051,188.189 57.9501,188.189 62.5951,188.189 67.2401,188.189 71.8851,188.189 76.5301,188.189 81.1751,188.189 
 85.8202,188.189 90.4652,188.189 95.1102,188.189 99.7552,188.189 104.4,188.189 109.045,188.189 113.69,188.189 118.335,188.189 122.98,188.189 127.625,188.189 
 132.27,188.189 136.915,188.189 141.56,188.189 146.205,188.189 150.85,188.189 155.495,188.189 160.14,188.189 164.785,188.189 169.43,188.189 174.075,188.189 
 178.72,188.189 183.365,188.189 188.01,188.189 192.655,188.189 197.3,188.189 201.945,188.189 206.59,188.189 211.235,188.189 215.88,188.189 220.525,188.189 
 225.17,188.189 229.815,188.189 234.46,188.189 239.105,188.189 243.75,188.189 248.395,188.189 253.04,188.189 257.685,188.189 262.33,188.189 266.975,188.189 
 271.62,188.189 276.265,188.189 280.91,188.189 285.555,188.189 290.2,188.189 294.845,188.189 299.491,188.189 304.136,188.189 308.781,188.189 313.426,188.189 
 318.071,188.189 322.716,188.189 327.361,188.189 332.006,188.189 336.651,188.189 341.296,188.189 345.941,188.189 350.586,188.189 355.231,188.189 359.876,188.189 
 364.521,188.189 369.166,188.189 373.811,188.189 378.456,188.189 383.101,188.189 387.746,188.189 392.391,188.189 397.036,188.189 401.681,188.189 406.326,188.189 
 410.971,188.189 415.616,188.189 420.261,188.189 424.906,188.189 429.551,188.189 434.196,188.189 438.841,188.189 443.486,188.189 448.131,188.189 452.776,188.189 
 457.421,188.189 462.066,188.189 466.711,188.189 471.356

In [9]:
plot(sM,ab[:,1],color=:black,linewidth=2,label="a")
plot!(sM,ab[:,2],color=:red,linewidth=2,label="b")
title!("Vasicek model: a and b in y(m) = a(m) + b(m)r, (mu,rho) = ($μ,$ρ)")
xlabel!("maturity (months)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 Vasicek model: a and b in y(m) = a(m) + b(m)r, (mu,rho) = (0.05,0.975) 
 
 
 maturity (months) 
 
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,368.504 44.0151,368.279 48.6601,368.057 53.3051,367.839 57.9501,367.625 62.5951,367.414 67.2401,367.207 71.8851,367.003 76.5301,366.802 81.1751,366.605 
 85.8202,366.411 90.4652,366.22 95.1102,366.032 99.7552,365.848 104.4,365.666 109.045,365.487 113.69,365.311 118.335,365.138 122.98,364.968 127.625,364.801 
 132.27,364.636 136.915,364.474 141.56,364.314 146.205,364.157 150.85,364.003 155.495,363.85 160.14,363.701 164.785,363.553 169.43,363.408 174.075,363.266 
 178.72,363.125 183.365,362.987 188.01,362.851 192.655,362.716 197.3,362.584 201.945,362.454 206.59,362.327 211.235,362.201 215.88,362.076 220.525,361.954 
 225.17,361.834 229.815,361.716 234.46,361.599 239.105,361.484 243.75,361.371 248.395,361.259 253.04,361.15 257.685,361.041 262.33,360.935 266.975,360.83 
 271.62,360.727 276.265,360.625 280.91,360.524 285.555,360.425 290.2,360.328 294.845,360.232 299.491,360.137 304.136,360.044 308.781,359.952 313.426,359.862 
 318.071,359.773 322.716,359.685 327.361,359.598 332.006,359.513 336.651,359.428 341.296,359.345 345.941,359.264 350.586,359.183 355.231,359.103 359.876,359.025 
 364.521,358.948 369.166,358.872 373.811,358.796 378.456,358.722 383.101,358.649 387.746,358.577 392.391,358.506 397.036,358.436 401.681,358.367 406.326,358.299 
 410.971,358.231 415.616,358.165 420.261,358.1 424.906,358.035 429.551,357.971 434.196,357.909 438.841,357.847 443.486,357.785 448.131,357.725 452.776,357.666 
 457.421,357.607 462.066,357.549 466.711,357.492 471.356,357.435 476.001,357.379 480.646,357.324 485.291,357.27 489.936,357.217 494.581,357.164 499.226,357.112 
 503.871,357.06 508.516,357.009 513.161,356.959 517.806,356.909 522.451,356.86 527.096,356.812 531.741,356.764 536.386,356.717 541.031,356.671 545.676,356.625 
 550.321,356.579 554.966,356.534 559.611,356.49 564.256,356.446 568.901,356.403 573.546,356.36 578.191,356.318 582.836,356.277 587.481,356.236 592.126,356.195 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 44.0151,12.3819 48.6601,16.8146 53.3051,21.1737 57.9501,25.4603 62.5951,29.676 67.2401,33.822 71.8851,37.8997 76.5301,41.9103 81.1751,45.855 
 85.8202,49.7351 90.4652,53.5518 95.1102,57.3063 99.7552,60.9996 104.4,64.633 109.045,68.2076 113.69,71.7243 118.335,75.1844 122.98,78.5888 127.625,81.9385 
 132.27,85.2346 136.915,88.4781 141.56,91.6698 146.205,94.8108 150.85,97.902 155.495,100.944 160.14,103.938 164.785,106.885 169.43,109.786 174.075,112.641 
 178.72,115.452 183.365,118.219 188.01,120.943 192.655,123.624 197.3,126.264 201.945,128.863 206.59,131.422 211.235,133.942 215.88,136.423 220.525,138.867 
 225.17,141.272 229.815,143.642 234.46,145.975 239.105,148.273 243.75,150.537 248.395,152.766 253.04,154.962 257.685,157.125 262.33,159.256 266.975,161.355 
 271.62,163.422 276.265,165.46 280.91,167.467 285.555,169.444 290.2,171.392 294.845,173.312 299.491,175.204 304.136,177.068 308.781,178.905 313.426,180.716 
 318.071,182.5 322.716,184.259 327.361,185.992 332.006,187.7 336.651,189.384 341.296,191.044 345.941,192.681 350.586,194.294 355.231,195.884 359.876,197.452 
 364.521,198.998 369.166,200.522 373.811,202.025 378.456,203.507 383.101,204.969 387.746,206.41 392.391,207.831 397.036,209.233 401.681,210.616 406.326,211.979 
 410.971,213.324 415.616,214.651 420.261,215.96 424.906,217.251 429.551,218.524 434.196,219.781 438.841,221.02 443.486,222.243 448.131,223.45 452.776,224.641 
 457.421,225.816 462.066,226.975 466.711,228.12 471.356,229.249 476.001,230.363 480.646,231.463 

# Hedging Using the Vasicek Model

Recall that the change of the (value of the) hedge portfolio is 

$\begin{equation}
\Delta V=h\Delta P_{H}-\Delta P_{L}
\end{equation}$


1. For an initial value of the short log interest rate $r$, use the Vasicek model to calculate all spot rates $y(m)$ for $m$ starting at 1 and going until the maturity of the liability or hedge (whichever is longest).

2. Use the spot interest rates ($y(m)$) to calculate the prices of both the liability and the hedge bond.

3. Redo points 1 and 2, but starting from a somewhat different short rate.
    
4. Calculate the $h$ value that makes $\Delta V = 0$, that is, $h=\Delta P_{L}/\Delta P_{H}$.

Hint: to transform from log rates $y(m)$ to effective rates $Y(m)$, use $Y(m) = \exp[y(m)]-1$.

### From Chapter on Bonds 1

In [10]:
function BondPrice3Ps(Y,cf,mk)              #cf is a vector of all cash flows, mk are years
    cfac = cf./((1+Y).^mk)                  #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P    = sum(cfac)                        #price
    return P
end

BondPrice3Ps (generic function with 1 method)

For given parameters of the Vasicek model ($\rho,\mu$), calculate bond prices at the short rate is $r_0$ and then at $r_1$. You may want to repeat the calculations but at different values of $\rho$

In [11]:
ρ = 0.9999                       #EXPERIMENT WITH CHANGING THIS
μ = 0.05

cfL = repeat([zeros(11);0.2],outer=10)   #cash flow of liability, MONTHLY
mkL = 1:120                              #times of the cash flows
cfH = [zeros(35);1]                      #cash flow of hedge bond
mkH = 1:36

r₀ = 0.05                         #initial (day 0) short log interest rate
r₁ = 0.049                        #after change (day 1)

y₀ = fill(NaN,length(mkL))        #yield curve, before
y₁ = fill(NaN,length(mkL))        #yield curve, after
for m in mkL                        
    ym,   = Vasicek([r₀ r₁],ρ,μ,m)
    y₀[m] = ym[1]
    y₁[m] = ym[2]
end    

PL₀ = BondPrice3Ps(exp.(y₀)-1,cfL,mkL/12)       #liability, before, /12 to get years
PL₁ = BondPrice3Ps(exp.(y₁)-1,cfL,mkL/12)       #after 
ΔPL = PL₁ - PL₀
printlnPs("PL₀, PL₁, ΔPL: ",[PL₀ PL₁ ΔPL])

PH₀ = BondPrice3Ps(exp.(y₀[mkH])-1,cfH,mkH/12)                   #hedge bond
PH₁ = BondPrice3Ps(exp.(y₁[mkH])-1,cfH,mkH/12)                   
ΔPH = PH₁ - PH₀
printlnPs("\nPH₀, PH₁, ΔPH: ",[PH₀ PH₁ ΔPH])

h = ΔPL/ΔPH
hdollars = h*PH₀/PL₀

printlnPs("\nh, dollars in hedge bond/value of liability: ", h," ",hdollars)

PL₀, PL₁, ΔPL:      1.535     1.543     0.008

PH₀, PH₁, ΔPH:      0.861     0.863     0.003

h, dollars in hedge bond/value of liability:      3.024      1.696
